<h1> Demo. Optimal Interpolation </h1>

An example of simulated SSH data access is provided in the "example_data_access.ipynb" notebook. Here, an example of simple mapping technique  is proposed. The notebook is structured as follow: 1) reading of pseudo-observations, 2) make reconstruction grid, 3) set optimal interpolation parameters and 4) perform optimal interpolation & save results

In [ ]:
import xarray as xr
import cftime
import geoviews as gv
import matplotlib.pylab as plt
import numpy as np
from datetime import datetime, timedelta
import numpy
#import pyinterp
#import holoviews as hv

#gv.extension('bokeh')

### 1) Read Pseudo-Observation

##### Read observations lon/lat/time/ssh

In [ ]:
# Ref start date of NATL60 simulation
simu_start_date = '2012-10-01'
def preprocess_time_swot_karin(ds):
    # This preprocessing must be avoided in future dataset release
    ds['time'] = cftime.num2date(ds['time'].values, 'seconds since ' + simu_start_date)
    # Duplicate time over across track dimension for swath
    ds = ds.stack(z=('nC', 'time'))
    return ds

ds_swot_karin = xr.open_mfdataset('./dc_obs/swot/BOOST-SWOT_SWOT_GULFSTREAM_c*.nc', preprocess=preprocess_time_swot_karin, combine='nested', concat_dim='z')
ds_swot_karin

In [ ]:
lon = ds_swot_karin.lon.values
lat = ds_swot_karin.lat.values
time = cftime.date2num(ds_swot_karin.time.values , units='days since 2012-10-01')
ssh = ds_swot_karin.ssh_model.values

##### Clean nan values

In [ ]:
indsel = np.where((np.isnan(ssh)==False))
lon = lon[indsel]
lat = lat[indsel]
time = time[indsel]
ssh = ssh[indsel]

skip = 100 
# Discard a bit ...
# ... for a "reasonable" computational time
lonr = lon[::skip]
latr = lat[::skip]
timer = time[::skip]
sshr = ssh[::skip]

#### 2) Define reconstruction grid

In [ ]:
# Grid
lon_min = 295.
lon_max = 305.
lat_min = 33.
lat_max = 43.
time_min = datetime.strptime('2012-10-01', '%Y-%m-%d')
time_max = datetime.strptime('2012-10-10', '%Y-%m-%d')

# box = [295., 305., 33., 43., 50., 60.]   # study box
dx = 0.2                                 # zonal spatial step (in degree)
dy = 0.2                                 # zonal spatial step (in degree)
dt = 1.                                  # meridional spatial step


# Make ouput grid lon/lat/time vector
glon = numpy.arange(lon_min, lon_max + dx, dx)
glat = numpy.arange(lat_min, lat_max + dy, dy)
gtime = numpy.arange(cftime.date2num(time_min, f'days since {simu_start_date}'), cftime.date2num(time_max, f'days since {simu_start_date}') + dt, dt)
nx = len(glon)
ny = len(glat)
nt = len(gtime)

# define & initialize ssh array
gssh = numpy.empty((nt,ny,nx))

# Make 2D grid
glon2, glat2 = numpy.meshgrid(glon, glat)
fglon = glon2.flatten()
fglat = glat2.flatten()
ng = len(fglat) # number of grid points

#### 3) Define Optimal Interpolation parameters

In [ ]:
Lx = 1.       # Zonal correlation scale in degree    
Ly = 1.       # Meridional correlation scale in degree
Lt = 7.       # Temporal correlation scale in days     
noise = 0.05  # noise level (% of signal variance)

#### 4) Perform Optimal interpolation

##### OI block (may take sevral minutes for 1 month OI)

In [ ]:
nobs_it = np.zeros(gtime.size)

for it in range(len(gtime)):
    
    ind1 = numpy.where((numpy.abs(timer-gtime[it])<2.*Lt))[0]
    nobs = len(ind1)
    print(it, '/', len(gtime)-1, 'nobs = ', nobs, end="\r")
    
    BHt = numpy.empty((ng,nobs))
    HBHt = numpy.empty((nobs,nobs))
    
    for iobs in range(nobs):
        BHt[:,iobs] = numpy.exp(-((gtime[it]-timer[ind1][iobs])/Lt)**2 - ((fglon-lonr[ind1][iobs])/Lx)**2 - ((fglat-latr[ind1][iobs])/Ly)**2)
        HBHt[:,iobs] = numpy.exp(-((timer[ind1]-timer[ind1][iobs])/Lt)**2 - ((lonr[ind1]-lonr[ind1][iobs])/Lx)**2 - ((latr[ind1]-latr[ind1][iobs])/Ly)**2)
        
    R = numpy.diag(numpy.full((nobs), noise**2))

    Coo = HBHt + R
    
    Mi = numpy.linalg.inv(Coo)

    sol = numpy.dot(numpy.dot(BHt, Mi), sshr[ind1])

    gssh[it,:,:] = sol.reshape(ny, nx)
    nobs_it[it] = nobs

##### Save reconstruction to dataset

In [ ]:
dc_reconstruction = xr.Dataset({'ssh_rec' : (('time', 'lat', 'lon'), gssh), 'nobs' : ('time', nobs_it)},
                               coords={'time': [numpy.datetime64(simu_start_date) + numpy.timedelta64(int(tt), 'D') for tt in gtime],
                                       'lon': glon - 360., 
                                       'lat': glat, 
                                       })  
dc_reconstruction                               

In [ ]:
### add save dc_reconstruction to netcdf
stmin = time_min.strftime('%Y-%m-%d')
stmax = time_max.strftime('%Y-%m-%d')
output_filename = f"ssh_rec_{stmin}_{stmax}.nc"
dc_reconstruction.to_netcdf(output_filename)

##### Plot example

In [ ]:
dc_ref = xr.open_mfdataset('./dc_ref/*.nc', combine='nested', concat_dim='time')
dc_ref

In [ ]:
time_selection = '2012-10-05T23:00:00'
plt.figure(figsize=(15, 5))
plt.subplot(121)
dc_reconstruction.ssh_rec.sel(time=time_selection, method='nearest').plot(vmin=-0.2, vmax=1, cmap='gist_stern')
plt.title('Reconstruction SSH', fontweight='bold')
plt.subplot(122)
dc_ref.sossheig.sel(time=time_selection, method='nearest').plot(vmin=-0.2, vmax=1, cmap='gist_stern')
plt.title('Reference SSH', fontweight='bold')
plt.savefig('example_ssh_ref_vs_ssh_reconstruction.png')